# Convolutional Neural Network (CNN)

- This Example demonstrates training a simple **Convolutional Neural Network(CNN)** to classify [CIFAR images](https://www.cs.toronto.edu/~kriz/cifar.html).
- The CIFAR10 dataset contains 60,000 color images in 10 classes, with 6,000 images in each class. The dataset is divided into 50,000 training images and 10,000 testing images. The classes are mutually exclusive and there is no overlap between them.


<img src="https://paperswithcode.com/media/datasets/CIFAR-10-0000000431-b71f61c0_U5n3Glr.jpg" width=70%>

In [3]:
from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.losses import SparseCategoricalCrossentropy

import seaborn as sns

from tensorflow.keras.optimizers import Adam

from keras_tuner.tuners import RandomSearch

from sklearn.metrics import confusion_matrix, classification_report

## Download and prepare the CIFAR10 dataset

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [5]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size = 0.80, random_state = 100)

In [6]:
# Normalize pixel values to be between 0 and 1
x_train = x_train / 255.0
x_val = x_val / 255.0
x_test = x_test / 255.0

In [7]:
y_train = y_train.reshape(-1,)
y_val = y_val.reshape(-1,)
y_test = y_test.reshape(-1,)

In [8]:
x_train.ndim

4

In [9]:
x_train.shape

(40000, 32, 32, 3)

In [10]:
x_train[0]

array([[[0.54117647, 0.54901961, 0.55294118],
        [0.29411765, 0.29411765, 0.2745098 ],
        [0.2627451 , 0.25490196, 0.25098039],
        ...,
        [0.9372549 , 0.9372549 , 0.9372549 ],
        [0.9372549 , 0.9372549 , 0.9372549 ],
        [0.94509804, 0.94509804, 0.94509804]],

       [[0.29411765, 0.30980392, 0.24313725],
        [0.22745098, 0.23529412, 0.16470588],
        [0.25490196, 0.25098039, 0.20784314],
        ...,
        [0.92156863, 0.92156863, 0.92156863],
        [0.92156863, 0.92156863, 0.92156863],
        [0.92156863, 0.92156863, 0.92156863]],

       [[0.19607843, 0.19607843, 0.12941176],
        [0.20392157, 0.20392157, 0.1254902 ],
        [0.23137255, 0.22745098, 0.14901961],
        ...,
        [0.77254902, 0.77254902, 0.77647059],
        [0.74901961, 0.74901961, 0.74901961],
        [0.74509804, 0.74509804, 0.74509804]],

       ...,

       [[0.51764706, 0.47843137, 0.50980392],
        [0.50980392, 0.47058824, 0.50196078],
        [0.50588235, 0

In [11]:
y_train

array([7, 5, 4, ..., 5, 4, 8], dtype=uint8)

In [12]:
y_train

array([7, 5, 4, ..., 5, 4, 8], dtype=uint8)

## Verify the data

To verify that the dataset looks correct, let's plot some images from the training set and display the class name below each image.

In [13]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

def plot_sample(x, y, index):
    plt.figure(figsize = (15,2))
    plt.imshow(x[index])
    plt.xlabel(class_names[y[index]])
    plt.show

In [14]:
plot_sample(x_train, y_train, 40)

## Convolutional Neural Network
<img src="https://miro.medium.com/max/2000/1*vkQ0hXDaQv57sALXAJquxA.jpeg">

### Create the convolutional base (Feature Extraction)

The 6 lines of code below define the convolutional base using a common pattern: a stack of [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) and [MaxPooling2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D) layers.

As input, a CNN takes tensors of shape (image_height, image_width, color_channels), ignoring the batch size. If you are new to these dimensions, color_channels refers to (R,G,B). In this example, you will configure your CNN to process inputs of shape (32, 32, 3), which is the format of CIFAR images. You can do this by passing the argument `input_shape` to your first layer.


In [15]:
model = Sequential()

In [16]:
model.add(Conv2D(filters = 50, kernel_size = (3,3), activation = 'relu', input_shape = (32,32,3)))
model.add(MaxPooling2D((2,2)))

In [17]:
model.add(Conv2D(filters = 100, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPooling2D((2,2)))

Let's display the summary of your model so far

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 50)        1400      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 50)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 100)       45100     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 100)         0         
Total params: 46,500
Trainable params: 46,500
Non-trainable params: 0
_________________________________________________________________


Above, you can see that the output of every Conv2D and MaxPooling2D layer is a 3D tensor of shape (height, width, channels). The width and height dimensions tend to shrink as you go deeper in the network. The number of output channels for each Conv2D layer is controlled by the first argument (e.g., 32 or 64). Typically,  as the width and height shrink, you can afford (computationally) to add more output channels in each Conv2D layer.

### Add Dense layers on top (Classification)

To complete the model, you will feed the last output tensor from the convolutional base (of shape (4, 4, 64)) into one or more Dense layers to perform classification. Dense layers take vectors as input (which are 1D), while the current output is a 3D tensor. First, you will flatten (or unroll) the 3D output to 1D,  then add one or more Dense layers on top. CIFAR has 10 output classes, so you use a final Dense layer with 10 outputs.

In [19]:
#Here's the complete architecture of your model:
model.add(Flatten())
model.add(Dense(100, activation = 'relu'))
model.add(Dense(10, activation = 'softmax'))

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 50)        1400      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 50)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 100)       45100     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 100)         0         
_________________________________________________________________
flatten (Flatten)            (None, 3600)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               360100    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1

The network summary shows that (4, 4, 64) outputs were flattened into vectors of shape (1024) before going through two Dense layers.

## Compile and train the model

In [21]:
model.compile(optimizer = 'adam', loss = SparseCategoricalCrossentropy(from_logits = True),
             metrics = ['accuracy'])

In [22]:
%time history = model.fit(x_train, y_train, epochs = 30, validation_data = (x_val, y_val))

Epoch 1/30
1250/1250 [==============================] - 8s 7ms/step - loss: 2.0939 - accuracy: 0.3616 - val_loss: 2.0033 - val_accuracy: 0.4562
Epoch 2/30
1250/1250 [==============================] - 6s 5ms/step - loss: 1.9732 - accuracy: 0.4855 - val_loss: 1.9188 - val_accuracy: 0.5413
Epoch 3/30
1250/1250 [==============================] - 6s 5ms/step - loss: 1.9102 - accuracy: 0.5494 - val_loss: 1.8960 - val_accuracy: 0.5636
Epoch 4/30
1250/1250 [==============================] - 7s 5ms/step - loss: 1.8722 - accuracy: 0.5878 - val_loss: 1.8635 - val_accuracy: 0.5963
Epoch 5/30
1250/1250 [==============================] - 7s 5ms/step - loss: 1.8450 - accuracy: 0.6144 - val_loss: 1.8557 - val_accuracy: 0.6010
Epoch 6/30
1250/1250 [==============================] - 7s 5ms/step - loss: 1.8256 - accuracy: 0.6350 - val_loss: 1.8500 - val_accuracy: 0.6075
Epoch 7/30
1250/1250 [==============================] - 7s 6ms/step - loss: 1.8057 - accuracy: 0.6565 - val_loss: 1.8454 - val_accuracy:

In [ ]:
history.history.keys()

## Evaluate the model

In [ ]:
plt.plot(history.history['accuracy'], label = 'accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc = 'lower right')

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose = 2)

In [ ]:
print(test_acc)

# Model-building function
Define a model-building function. It takes an argument `hyperparameter` from which you can sample hyperparameters, such as `hyperparameter.Int('units', min_value=32, max_value=512, step=32)` (an integer from a certain range).

This function returns a compiled model.